In [150]:
import numpy as np
import random

import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,RepeatedStratifiedKFold, cross_val_score, cross_validate
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import  accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
import os
import matplotlib.pyplot as plt
import shutil
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt  
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2,f_classif
from sklearn import preprocessing
import math
from sklearn.metrics import fbeta_score
from numpy import arange
from sklearn.metrics import balanced_accuracy_score

In [151]:
metric_to_optimize = 'MCC'

In [152]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=RuntimeWarning)

In [153]:
def print_and_summarize_best_model(results,ml_results,best_models):
    ml_results = pd.concat([ml_results, results],sort=True)
    best_model_index = np.argmax(np.array(results[metric_to_optimize]))
    best_model = results.iloc[best_model_index]
    best_models = pd.concat([best_models, pd.DataFrame(best_model).transpose()],sort = True)
    
    return ml_results,best_models,best_model_index

In [154]:
def init_metrics():
    '''
    initialize the (empty) metrics structures
    :return: metrics_results
    '''
    metrics_results = {'FNR': None, 'FPR': None, 'MCC':None, 'ACC': None, 'F1score': None,'F2score': None}
    metrics_results['FNR'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    metrics_results['FPR'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    metrics_results['MCC'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    metrics_results['ACC'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    metrics_results['F1score'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    metrics_results['F2score'] = {'DT':None,'SVM':None,'RF':None,'NB': None,'KNN':None,'LogisticRegression':None}
    return metrics_results

def init_best_models():
    best_models = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    return best_models

def init_ml_results():
    ml_results = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    return ml_results

In [155]:
def metrics_calculation_as_binary(all_y,all_predictions):
    cm = confusion_matrix(all_y,all_predictions,labels = ['Clog','Control','Leak'])
    TP = cm[0,0]+cm[0,2]+cm[2,2]+cm[2,0] # clog or leak classified correctly
    FN = cm[0,1]+cm[2,1] #clog or leak classified as control
    FP = cm[1,0]+cm[1,2] #control classified as leak or clog
    TN = cm[1,1] #control  classified correctly
    
    FNR = FN/(FN+TP)
    FPR = FP/(FP+TN)

    Recall =TP/(TP+FN)
    Precision=TP/(TP+FP)
#     Accuracybinary = (TP+TN)/(TP+TN+FN+FP)
    F1_Binary = 2*(Recall*Precision)/(Recall+Precision)
    F2_Binary = ((1 + 2**2) * Precision * Recall) / (2**2 * Precision + Recall)
    return FNR,FPR,F1_Binary,F2_Binary

In [156]:
def calculate_metrics(classifier_name,y_test,y_prediction_old,metrics_results):
    '''
    calculate each metric for evaluation
    :param classifier_name: name of classifier used (svm, NB, etc)
    :param y_test:
    :param y_prediction:
    :param metrics_results:
    :return:
    '''
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report

    from sklearn.metrics import matthews_corrcoef
    y_test = y_test.astype('category').to_numpy()
    y_prediction = np.copy(y_prediction_old)
    y_prediction[(y_test == 'Clog') & (y_prediction_old=="Leak")]='Clog' #CLOG CLASIFIED AS LEAK
    y_prediction[(y_test == 'Leak') & (y_prediction_old=="Clog")]='Leak'# LEAK CLASSIFIED AS CLOG
    
    cnf_matrix = confusion_matrix(y_test, y_prediction,
                                         labels=["Leak", "Clog", "Control"])
#     import seaborn as sns
#     plt.figure()
#     sns.heatmap(cnf_matrix, annot=True)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)

    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Fall out or false positive rate
    FPR_mal = (FP/(FP+TN))

    # False negative rate
    FNR_mal = (FN/(TP+FN))

    y_prediction_binary = np.zeros(y_test.size)
    y_prediction_binary[(y_prediction_old=="Clog") | (y_prediction_old=="Leak")]=1 #CLOG CLASIFIED AS LEAK
    y_prediction_binary[y_prediction_old == "Control"]=0
    
    y_test_binary =np.zeros(y_test.size)
    y_test_binary[(y_test == "Clog")| (y_test=="Leak")]=1
    y_test_binary[y_test == "Control"]=0
    

    TN, FP, FN, TP = confusion_matrix(y_test_binary, y_prediction_binary).ravel()

    FPR  = FP / (FP + TN)
    FNR = FN / (FN + TP)

    F1_Binary = fbeta_score(y_test_binary, y_prediction_binary, beta=1)
    F2_Binary = fbeta_score(y_test_binary, y_prediction_binary, beta=2)
    MCC = matthews_corrcoef(y_test_binary, y_prediction_binary)
    Accuracy = accuracy_score(y_test_binary, y_prediction_binary)
    balanced_ACC = balanced_accuracy_score(y_test_binary, y_prediction_binary)
    
#     FNR,FPR,F1_Binary,F2_Binary = metrics_calculation_as_binary(y_test, y_prediction)#As Binary
    
    MCC_multiclass = matthews_corrcoef(y_test, y_prediction)
#     Accuracy = accuracy_score(y_test, y_prediction)
    metrics_results['FNR'][classifier_name] = {'FNR':FNR,'Leak':FNR_mal[0],'Clog':FNR_mal[1],'MCC_multiclass':MCC_multiclass}
    metrics_results['FPR'][classifier_name] = {'FPR':FPR,'Leak':FPR_mal[0],'Clog':FPR_mal[1],'balanced_ACC':balanced_ACC}
    metrics_results['MCC'][classifier_name] = MCC
    metrics_results['ACC'][classifier_name] = Accuracy
    metrics_results['F1score'][classifier_name] = F1_Binary
    metrics_results['F2score'][classifier_name] = F2_Binary
    return metrics_results

In [178]:
def save_folds_for_best_model(model_index,folds,MCC_results,F1score_results,F2score_results,FNR_results):
    print(model_index)
    folder_path = 'C:\\Users\\noy\\Desktop\\תזה\\Results\\Kedma_secend_best_model_'
    balanced_ACC_results = pd.DataFrame(d['balanced_ACC'] for d in FNR_results).values.ravel()
    Al = 'KNN_73_multiclass'

    best_mudel_res= pd.DataFrame(data={'Algorithm':Al,
                                          'Fold_number':folds,
                                          'MCC': MCC_results,
                                           'balanced_ACC':balanced_ACC_results,
                                          'F1score': F1score_results,
                                          'F2score': F2score_results})
    best_mudel_res =best_mudel_res.set_index('Fold_number')
    best_mudel_res.to_csv(folder_path+Al+'_folds_results.csv')
    print('Created')
    return

In [177]:
def CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params):
    folds,FNR_results, FPR_results, MCC_results, ACC_results,F1score_results,F2score_results = [],[],[],[],[],[],[]                      
    if(model_index==73):
        random.seed(10)
        fold=1
        skf = RepeatedStratifiedKFold(n_splits=5,n_repeats=5, random_state=12)
        # K_folds:
        for train_index, test_index in skf.split(data, data_labels):
            x_train_fold, x_test_fold = data.iloc[train_index], data.iloc[test_index]
            y_train_fold, y_test_fold = data_labels.iloc[train_index], data_labels.iloc[test_index]
            if (Algorithm not in ['RF','DT']):
                if (Algorithm in('Naive Bayes','svm_linear')):
                    scaler = MinMaxScaler()
                else:
                    scaler = StandardScaler()
                x_train_fold = scaler.fit_transform(x_train_fold)
                x_test_fold = scaler.fit_transform(x_test_fold)
            classifier.fit(x_train_fold, y_train_fold.values.ravel())
            y_prediction = classifier.predict(x_test_fold)
            metrics_results = calculate_metrics(Algorithm,y_test_fold,y_prediction,metrics_results)
            FNR_results.append(metrics_results['FNR'][Algorithm])
            FPR_results.append(metrics_results['FPR'][Algorithm])
            MCC_results.append(metrics_results['MCC'][Algorithm])
            ACC_results.append(metrics_results['ACC'][Algorithm])
            F1score_results.append(metrics_results['F1score'][Algorithm])
            F2score_results.append(metrics_results['F2score'][Algorithm])
            folds.append(fold)
            fold =fold+1
        save_folds_for_best_model(model_index,folds,MCC_results,F1score_results,F2score_results,FPR_results)

        results_summary = results_summary.append({'Algorithm':Algorithm,
                                                  'model_index':model_index,
                                                  'parameters':params,
                                                  'FNR': {'FNR':sum(d['FNR'] for d in FNR_results) / len(FNR_results),
                                                         'FNR_Leak':sum(d['Leak'] for d in FNR_results) / len(FNR_results),
                                                         'FNR_Clog':sum(d['Clog'] for d in FNR_results) / len(FNR_results),
                                                         'MCC_multiclass':sum(d['MCC_multiclass'] for d in FNR_results) / len(FNR_results)}, #np.mean(FNR_results),
                                                  'FPR': {'FPR':sum(d['FPR'] for d in FPR_results) / len(FPR_results),
                                                         'FPR_Leak':sum(d['Leak'] for d in FPR_results) / len(FPR_results),
                                                         'FPR_Clog':sum(d['Clog'] for d in FPR_results) / len(FPR_results),
                                                          'balanced_ACC':sum(d['balanced_ACC'] for d in FPR_results) / len(FPR_results)},
                                                  'MCC': np.mean(MCC_results),
                                                  'ACC': np.mean(ACC_results),
                                                  'F1score': np.mean(F1score_results),
                                                  'F2score': np.mean(F2score_results)},
                                                 ignore_index=True)
    return results_summary

In [159]:
def train_test_CV_RandomForest(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'RF' 
    
    n_estimators = [200, 500]               # number of trees in the foreset
    max_depth = [5, 8, 10, 20]              # max number of levels in each decision tree
    criterion = ['gini', 'entropy']
    min_samples_split = [2, 5, 10, 15,20]   # min number of data points placed in a node before the node is split, used to control over-fitting
    bootstrap = [True, False]
    for n in n_estimators:
        for max in max_depth:
            for c in criterion:
                for min_sample in min_samples_split:
                    for b in bootstrap:
                        classifier = RandomForestClassifier(random_state=12,class_weight='balanced', n_estimators=n,
                                                            max_depth=max, criterion=c,min_samples_split=min_sample,bootstrap=b)  # class_weight: misclassifying the rare class is punished harsher.
                        params = {"n_estimators": n,"criterion": c,"min_samples_split":min_sample,"bootstrap":b,'max_depth':max}
                        results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)
                        model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results

In [160]:
def train_test_CV_svm_linear(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'svm_linear'
    kernel = 'linear'
    cs = [0.1, 1, 10, 100]
    for c in cs:
        classifier = svm.SVC(probability=True,kernel=kernel,C=c,max_iter=100000)
        params = {"kernel": kernel,"C": c}
        results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)
        model_index+=1

    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results   

In [161]:
def train_test_CV_svm_rbf(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'svm_rbf'
    kernel =  'rbf'
    cs = [0.1, 1, 10, 100]
    gammas = [0.0001, 0.001, 0.01, 0.1, 1, 2, 3, 4]
    for c in cs:
        for gamma in gammas:
            classifier = svm.SVC(probability=True,kernel=kernel,C=c,gamma=gamma,max_iter=100000)
            params = {"kernel": kernel,"C": c,'gamma':gamma}
            results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)

            model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results   

In [162]:
def train_test_CV_svm_poly(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'svm_poly'

    kernel = 'poly'
    cs = [0.1, 1, 10, 100]
    gammas = [0.0001, 0.001, 0.01, 0.1, 1]
    degrees = [2,3,4,5]
    for c in cs:
        for gamma in gammas:
            for degree in degrees:
                classifier = svm.SVC(probability=True,kernel=kernel,C=c,gamma=gamma,degree=degree,max_iter=100000)
                params = {"kernel": kernel,"C": c,'gamma':gamma,'degree':degree}
                results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)

                model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results   

In [163]:
def train_test_CV_DecisionTree(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'DT'
    
    criterion = ["gini", "entropy"]
    splitter = ["best", "random"]
    min_samples_split = [2, 5, 10, 15,20]  # min number of data points placed in a node before the node is split, used to control over-fitting
    for c in criterion:
        for s in splitter:
            for min_split in min_samples_split:
                classifier = tree.DecisionTreeClassifier(criterion=c,splitter=s,min_samples_split=min_split)
                params = {"criterion": c,"splitter": s,'min_samples_split':min_split}
                results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)

                model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results   

In [164]:
def train_test_CV_NaiveBayes(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'NB'
    Algorithm = 'Naive Bayes'
    NB_Classifiers = [GaussianNB(), MultinomialNB()]
    # CategoricalNB()
    for NB_clf in NB_Classifiers:
        classifier = NB_clf  # create classifier
        params = {"NB_Classifiers": NB_clf}
        results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)

        model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results   

In [165]:
def train_test_CV_KNN(folder_path_results,data,data_labels,metrics_results,best_models, ml_results):
    model_index = int(1)
    results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                               'FNR','FPR','MCC','ACC','F1score','F2score'])
    Algorithm = 'KNN'
    n_neighbors = range(1, 11)
    weights = ['uniform', 'distance']
    Algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorithm used to compute the nearest neighbors
    for n in n_neighbors:
        for w in weights:
            for alg in Algorithms:
                classifier = KNeighborsClassifier(n_neighbors=n,weights=w,algorithm=alg)  # create classifier
                params = {"n_neighbors": n,'weights':w,"Algorithm":alg}
                results_summary = CV_model_tunning(data,data_labels,classifier,Algorithm,model_index,metrics_results,results_summary,params)

                model_index+=1
    print(Algorithm,'_results_summary was created')
    results_summary.to_csv(folder_path_results+'\\'+Algorithm+'_results_summary.csv')
    ml_results,best_models,best_model_index = print_and_summarize_best_model(results_summary,ml_results,best_models)
    return ml_results,best_models,metrics_results  

In [175]:
def conduct_tunning_and_classification_for_each_family_model_CV(data,data_labels,stage):
    best_models = init_best_models()
    ml_results = init_ml_results()
    metrics_results = init_metrics()
    folder_path = 'C:\\Users\\noy\\Desktop\\תזה\\Results\\Multiclass_calssification\\Kedma'+stage
    folder_path_results = folder_path+'\\classifiers'
#     print('DecisionTree')
#     ml_results,best_models,metrics_results = train_test_CV_DecisionTree(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
#     print('RandomForest')
#     ml_results,best_models,metrics_results = train_test_CV_RandomForest(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
#     print('NaiveBayes')
#     ml_results,best_models,metrics_results = train_test_CV_NaiveBayes(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
    print('KNN')
    ml_results,best_models,metrics_results = train_test_CV_KNN(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
#     print('svm_linear')
#     ml_results,best_models,metrics_results = train_test_CV_svm_linear(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
#     print('svm_rbf')
#     ml_results,best_models,metrics_results = train_test_CV_svm_rbf(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
#     print('svm_poly')
#     ml_results,best_models,metrics_results = train_test_CV_svm_poly(folder_path_results,data,data_labels,metrics_results,best_models, ml_results)
   
    best_models = pd.concat([best_models.drop(['FNR','FPR'], axis=1),
               best_models['FNR'].apply(pd.Series),
               best_models['FPR'].apply(pd.Series)], axis=1)
    ml_results = pd.concat([ml_results.drop(['FNR','FPR'], axis=1),
               ml_results['FNR'].apply(pd.Series),
               ml_results['FPR'].apply(pd.Series)], axis=1)
   
    best_models = best_models.reindex(columns=['Algorithm',
                                               'model_index',
                                               'MCC',
                                               'ACC',
                                              'F1score',
                                              'F2score',
                                              'FNR',
                                              'FPR',
                                              'FNR_Leak',
                                              'FNR_Clog',
                                              'FPR_Leak',
                                              'FPR_Clog',
                                               'balanced_ACC',
                                              'MCC_multiclass']).reset_index().drop(['index'],axis=1)
    
    best_models.iloc[:,2:] = best_models.iloc[:,2:].round(5)
    best_models =best_models.set_index('Algorithm')
    best_models=best_models.T
    ml_results = ml_results.sort_values(by=[metric_to_optimize],ascending = False)

#     best_models.to_csv(folder_path+'\\best_models_include_malf_metrics.csv')
#     ml_results.to_csv(folder_path+'\\ml_results_include_malf_metrics.csv')
    return ml_results,best_models,metrics_results

In [167]:
def RFECV_Random_Forest(X,y):
    from numpy import mean
    from numpy import std
    from sklearn.datasets import make_classification
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedStratifiedKFold
    from sklearn.feature_selection import RFECV
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import matthews_corrcoef
    import sklearn.metrics as m
    from sklearn.metrics import fbeta_score, make_scorer
    ftwo_scorer = make_scorer(fbeta_score, beta=2, average='weighted')

    average='weighted'
    # Automatically select the number of features for RFE
    # create pipeline
    rfe = RFECV(estimator=RandomForestClassifier())

    model = RandomForestClassifier()
    pipeline = Pipeline(steps=[('s',rfe),('m',model)])
    # evaluate model
    cv = RepeatedStratifiedKFold(n_splits=5,n_repeats=5, random_state=12)
    
    n_scores = cross_val_score(pipeline, X, y, scoring=ftwo_scorer, cv=cv, n_jobs=-1, error_score='raise')
    # report performance
    print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
    rfe.fit(X, y)
    
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score \n of number of selected features")
    plt.plot(range(1, len(rfe.grid_scores_)+1), rfe.grid_scores_)
    plt.show()

    print('The optimal number of features is {}'.format(rfe.n_features_))
    features = [f for f,s in zip(X.columns, rfe.support_) if s]
    print('The selected features are:')
    print ('{}'.format(features))
    
    for i in range(X.shape[1]):
        print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    
    return features

In [168]:
df_Lavi_Kedma = pd.read_excel(os.path.join(os.getcwd(), 'data_Kedma&Lavi_afer_DataPreparation.xlsx')).iloc[: , 1:] 
df_Lavi_Kedma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   RTK_Num_All_Plots_Data  393 non-null    int64  
 1   Plot_All_Plots_Data     393 non-null    object 
 2   Site_Name               393 non-null    object 
 3   min                     393 non-null    float64
 4   prctile5                393 non-null    float64
 5   prctile95               393 non-null    float64
 6   std                     393 non-null    float64
 7   MTD                     393 non-null    float64
 8   median                  393 non-null    float64
 9   skewness                393 non-null    float64
 10  IQR                     393 non-null    float64
 11  meanci_1                393 non-null    float64
 12  meanci_2                393 non-null    float64
 13  Diff_median_of_plot     393 non-null    float64
 14  Diff_std_of_plot        393 non-null    fl

In [169]:
df_Lavi_Kedma['Type']=df_Lavi_Kedma['Type'].astype('category')

data_train = df_Lavi_Kedma[df_Lavi_Kedma.Site_Name=='Kedma'].copy(True)
data_labels_train = data_train[['Type']]

data_test = df_Lavi_Kedma[df_Lavi_Kedma.Site_Name=='Lavi'].copy(True)
data_labels_test = data_test['Type']

vars_to_drop = ["RTK_Num_All_Plots_Data",'Plot_All_Plots_Data','Site_Name','SWP','Type','Y']

## Recursive featue elimination - cross validation based on RF

Selecting features using RFECV with RF and score F2score - as FN (miss malfunction is more important)

## Y as categorial

In [170]:
data_labels_train
data_train.drop(vars_to_drop,axis=1).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 204 to 392
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   min                     189 non-null    float64
 1   prctile5                189 non-null    float64
 2   prctile95               189 non-null    float64
 3   std                     189 non-null    float64
 4   MTD                     189 non-null    float64
 5   median                  189 non-null    float64
 6   skewness                189 non-null    float64
 7   IQR                     189 non-null    float64
 8   meanci_1                189 non-null    float64
 9   meanci_2                189 non-null    float64
 10  Diff_median_of_plot     189 non-null    float64
 11  Diff_std_of_plot        189 non-null    float64
 12  Diff_mean_of_row        189 non-null    float64
 13  Diff_std_of_row         189 non-null    float64
 14  prctile33               189 non-null    

In [171]:
random.seed(10)
# features = RFECV_Random_Forest(data_train.drop(vars_to_drop,axis=1),data_labels_train)

In [172]:
features=['min', 'prctile5', 'prctile95', 'std', 'MTD', 'median', 'skewness', 'IQR', 'meanci_1', 'Diff_median_of_plot', 'Diff_std_of_plot', 'Diff_mean_of_row', 'Diff_std_of_row', 'prctile33', 'median_first_ring', 'median_second_ring', 'median_core', 'CWSI', 'CWSI_minus_CWSI_of_row']

In [173]:
data_train[features]
data_labels_train =data_labels_train['Type']

In [179]:
ml_results,best_models,metrics_results = conduct_tunning_and_classification_for_each_family_model_CV(data_train[features],
                                                                                                     data_labels_train,
                                                                                                     stage='')

KNN
73
Created
KNN _results_summary was created


In [99]:
best_models#.sort_values(by=[metric_to_optimize],ascending=False )

Algorithm,DT,RF,Naive Bayes,KNN,svm_linear,svm_rbf,svm_poly
model_index,20,39,2,73,2,12,15
MCC,0.162229,0.283351,0.105837,0.302345,0.085634,0.208864,0.229525
ACC,0.636188,0.691181,0.658236,0.690185,0.633826,0.655164,0.539772
F1score,0.731371,0.774937,0.772957,0.767972,0.7475,0.745714,0.545343
F2score,0.734796,0.779582,0.821671,0.760022,0.776649,0.745335,0.453521
FNR,0.26191,0.2168,0.14117,0.24455,0.20178,0.25385,0.59194
FPR,0.58333,0.50667,0.77333,0.45,0.72,0.54,0.17667
FNR_Leak,0.33538,0.27795,0.14385,0.34513,0.2041,0.31487,0.6059
FNR_Clog,0.19077,0.15692,0.13846,0.14462,0.2,0.19385,0.57846
FPR_Leak,0.1424,0.1456,0.1968,0.096,0.2016,0.1488,0.048


### Saving folds results for T-test

In [147]:
params = {'n_neighbors': 10, 'weights': 'uniform', 'Algorithm': 'auto'}

model_index = 73
results_summary = pd.DataFrame(columns=['Algorithm','model_index', 'parameters',
                                           'FNR','FPR','MCC','ACC','F1score','F2score'])
data = data_train[features]
data_labels =data_labels_train


classifier = KNeighborsClassifier(n_neighbors=10,weights='uniform',algorithm='auto')  # create classifier
    
Fold_num, MCC_results,F1score_results,F2score_results,FPR_results = [],[],[],[],[]               
fold =1
random.seed(10)
skf = RepeatedStratifiedKFold(n_splits=5,n_repeats=5, random_state=12)
    # K_folds:
for train_index, test_index in skf.split(data, data_labels):
    x_train_fold, x_test_fold = data.iloc[train_index], data.iloc[test_index]
    y_train_fold, y_test_fold = data_labels.iloc[train_index], data_labels.iloc[test_index]
    if (Algorithm not in ['RF','DT']):
        if (Algorithm in('Naive Bayes','svm_linear')):
            scaler = MinMaxScaler()
        else:
            scaler = StandardScaler()
        x_train_fold = scaler.fit_transform(x_train_fold)
        x_test_fold = scaler.fit_transform(x_test_fold)
    classifier.fit(x_train_fold, y_train_fold.values.ravel())
    y_prediction = classifier.predict(x_test_fold)
    metrics_results = calculate_metrics(Algorithm,y_test_fold,y_prediction,metrics_results)
    Fold_num.append(fold)
    MCC_results.append(metrics_results['MCC'][Algorithm])
    FPR_results.append(metrics_results['FPR'][Algorithm])
    F1score_results.append(metrics_results['F1score'][Algorithm])
    F2score_results.append(metrics_results['F2score'][Algorithm])
    fold =fold+1
balanced_ACC_results = pd.DataFrame(d['balanced_ACC'] for d in FPR_results).values.ravel()
balanced_ACC_results

results_summary= pd.DataFrame(data={'Algorithm':Algorithm,
                                              'Fold_number':Fold_num,
                                              'MCC': MCC_results,
                                               'balanced_ACC':balanced_ACC_results,
                                              'F1score': F1score_results,
                                              'F2score': F2score_results})
results_summary =results_summary.set_index('Fold_number')
results_summary

results_summary.to_csv(folder_path+Algorithm+'_folds_results.csv')

## Evaluating model performence on independent data set (Lavi)

In [23]:
test_set  = df_Lavi_Kedma[df_Lavi_Kedma.Site_Name=='Lavi'].copy(True)

In [24]:
data_labels_test['Type']= test_set['Type'].astype('category').copy(True)
data_test = test_set[features]
data_labels_test['Type']

0      Control
1         Leak
2      Control
3      Control
4         Leak
        ...   
199       Leak
200    Control
201       Leak
202       Leak
203       Leak
Name: Type, Length: 204, dtype: category
Categories (3, object): ['Clog', 'Control', 'Leak']

In [120]:
results = init_metrics()
Algorithm='SVM rbf'
params = {
          "kernel": "rbf",
          "c": 0.1,
          "gamma": 0.001
        }
    
classifier=svm.SVC(probability=True,kernel=params['kernel'],C=params['c'],gamma=params['gamma'],max_iter=100000)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(data_train)
x_test= scaler.fit_transform(data_test)

classifier.fit(x_train, data_labels_train.values.ravel())
y_prediction = classifier.predict(x_test)

results = calculate_metrics(Algorithm,data_labels_test,y_prediction,results)

results_summary = pd.DataFrame([[Algorithm,*list(params.values()),
                                      results['FNR'][Algorithm], results['FPR'][Algorithm],
                                     results['MCC'][Algorithm], results['ACC'][Algorithm],
                                     results['F1score'][Algorithm], results['F2score'][Algorithm]]],
            columns=['Algorithm',*list(params.keys()),'FNR','FPR', 'MCC', 'ACC','F1score','F2score'])

results_summary

ValueError: could not convert string to float: 'B'

# Y as ordinary

In [100]:
Y_Ordinary = pd.DataFrame(data_labels_train).copy(True)
Y_Ordinary
Y_Ordinary['Y'] = 0 
Y_Ordinary.loc[Y_Ordinary['Type'] == 'Clog', 'Y'] = 0
Y_Ordinary.loc[Y_Ordinary['Type'] == 'Control', 'Y'] = 1
Y_Ordinary.loc[Y_Ordinary['Type'] == 'Leak', 'Y'] = 2

Y_Ordinary_new= pd.Series(Y_Ordinary['Type'], dtype="category")
Y_Ordinary_new= Y_Ordinary_new.cat.codes
X=data_train.drop(vars_to_drop,axis=1)
y=Y_Ordinary_new


In [1]:
import random
random.seed(10)
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef
import sklearn.metrics as m
import mord as mor

from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2, average='weighted')

average='weighted'
# Automatically select the number of features for RFE
# create pipeline
rfe = RFECV(estimator= mor.LogisticIT())

model =mor.LogisticIT()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=5,n_repeats=5, random_state=12)

n_scores = cross_val_score(pipeline, X, y, scoring=ftwo_scorer, cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
rfe.fit(X, y)

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score \n of number of selected features")
plt.plot(range(1, len(rfe.grid_scores_)+1), rfe.grid_scores_)
plt.show()

print('The optimal number of features is {}'.format(rfe.n_features_))
features2 = [f for f,s in zip(X.columns, rfe.support_) if s]
print('The selected features are:')
print ('{}'.format(features2))

for i in range(X.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))


ModuleNotFoundError: No module named 'mord'

In [102]:
features2 = ['Diff_std_of_row', 'CWSI_minus_CWSI_of_row']
data_train2=data_train[features2].copy(True)
data_train2

,Diff_std_of_row,CWSI_minus_CWSI_of_row
204,-0.020616,0.277926
205,-0.389405,0.255195
206,0.068012,0.179078
207,0.091777,0.170073
208,0.198757,0.037510
...,...,...
388,0.353332,-0.110986
389,-0.411500,0.372105
390,0.105174,-0.194867
391,0.193455,-0.057650


In [103]:
ml_results,best_models,metrics_results = conduct_tunning_and_classification_for_each_family_model_CV(data_train2,
                                                                                                     data_labels_train,
                                                                                                     stage='\\Features_of_Y_as_ordinary')

DecisionTree
DT _results_summary was created
RandomForest
RF _results_summary was created
NaiveBayes
Naive Bayes _results_summary was created
KNN
KNN _results_summary was created
svm_linear
svm_linear _results_summary was created
svm_rbf
svm_rbf _results_summary was created
svm_poly


C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Cons

C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Cons

C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Cons

C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Cons

C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\noy\Anaconda3\envs\seaversioneleven\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Cons

svm_poly _results_summary was created


In [104]:
best_models#.sort_values(by=['F2score'],ascending=False )

Algorithm,DT,RF,Naive Bayes,KNN,svm_linear,svm_rbf,svm_poly
model_index,10,19,1,73,2,28,77
MCC,0.155159,0.1986,0.105998,0.142241,0.122508,0.209649,0.26337
ACC,0.626686,0.638179,0.630612,0.625405,0.644324,0.63909,0.601422
F1score,0.719745,0.724251,0.739265,0.723228,0.753658,0.722206,0.645802
F2score,0.713468,0.711499,0.75819,0.719714,0.781258,0.704018,0.574139
FNR,0.28923,0.29557,0.22794,0.28215,0.19852,0.30665,0.46486
FPR,0.55333,0.50333,0.67333,0.57333,0.69333,0.47667,0.25667
FNR_Leak,0.33308,0.32436,0.22179,0.33154,0.17513,0.29641,0.52385
FNR_Clog,0.24615,0.26769,0.23385,0.23385,0.22154,0.31692,0.40615
FPR_Leak,0.1616,0.1824,0.1984,0.1584,0.2144,0.1632,0.0848


## Evaluating model performence on independent data set (Lavi)

In [65]:
data_test2=test_set[features2].copy(True)
data_test2.head(5)

,MTD,Meanci_1,Diff_median_of_plot,Diff_mean_of_row,Diff_std_of_row,median_second_ring,CWSI,CWSI_minus_CWSI_of_row
0,1.857778,3.180862,-1.293726,-1.275677,0.687112,2.991489,0.537787,-0.320373
1,9.267117,4.857138,-1.413829,-1.292092,0.497600,3.774675,0.617892,-0.234274
2,2.295044,2.880736,-0.966367,-0.964791,0.558871,2.636387,0.441409,-0.223995
3,2.695513,2.661326,-0.702693,-0.979400,0.629056,2.423534,0.388723,-0.213326
4,10.044307,4.111615,-0.962213,-1.359424,0.274137,3.275674,0.483383,-0.210300


In [ ]:
results2 = init_metrics()
{'kernel': 'rbf', 'C': 100, 'gamma': 0.1}

Algorithm='SVM rbf'
params = {
          "kernel": "rbf",
          "c": 0.1,
          "gamma": 0.001
        }
    
classifier=svm.SVC(probability=True,kernel=params['kernel'],C=params['c'],gamma=params['gamma'],max_iter=100000)

scaler = MinMaxScaler()
x_train2 = scaler.fit_transform(data_train2)
x_test2= scaler.fit_transform(data_test2)

classifier.fit(x_train2,data_labels_train2.values.ravel())
y_prediction2 = classifier.predict(x_test2)

results2 = calculate_metrics(Algorithm,data_labels_test2,y_prediction2,results2)

results_summary = pd.DataFrame([[Algorithm,*list(params.values()),
                                      results2['FNR'][Algorithm], results2['FPR'][Algorithm],
                                     results2['MCC'][Algorithm], results2['ACC'][Algorithm],
                                     results2['F1score'][Algorithm], results2['F2score'][Algorithm]]],
            columns=['Algorithm',*list(params.keys()),'FNR','FPR', 'MCC', 'ACC','F1score','F2score'])

results_summary